# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import cherrypy
import json
import redis
from datetime import datetime

## Connect to the Redis Database

In [ ]:
REDIS_HOST = 'redis-17291.c85.us-east-1-2.ec2.redns.redis-cloud.com'
REDIS_PORT = 17291
REDIS_USER = 'default'
REDIS_PASSWORD = '59IrBjnr5SLciwWOLQp7OZlAxAFlpCAo'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USER, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

## Instructions:
1) Create a class for each endpoint (status, sensors, sensor)
    - Status
    - Sensors
    - Sensor
2) For each endpoint, implement the required HTTP methods (GET, POST, PUT, DELETE)
    - Status: GET
    - Sensors: GET, POST
    - Sensor: GET, PUT, DELETE
3) Map each object to its target endpoint.
    - Status() -> "/status"
    - Sensors() -> "/sensors"
    - Sensor() -> "/sensor"

## Status Endpoint Class

In [3]:
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

## Sensors Endpoint Class

In [4]:
class Sensors(object):
    exposed = True

    def GET(self, *path, **query):
        # print(query)
        min_t_samples = int(query.get('min_t_samples', 0))
        min_h_samples = int(query.get('min_h_samples', 0))
        sensors = []
        keys = redis_client.keys('0x*:temperature')

        count = 0
        for key in keys:
            key = key.decode()
            mac_address = key.split(':')[0]

            t_info = redis_client.ts().info(f'{mac_address}:temperature')
            t_samples = t_info.total_samples
            t_retention = t_info.retention_msecs
            h_info = redis_client.ts().info(f'{mac_address}:humidity')
            h_samples = h_info.total_samples
            h_retention = h_info.retention_msecs
            
            if t_samples >= min_t_samples and h_samples >= min_h_samples:
                sensors.append(
                    {
                        "mac_address": mac_address,
                        "t_samples": t_samples,
                        "t_retention": t_retention,
                        "h_samples": h_samples,
                        "h_retention": h_retention,
                    }
                )
                count += 1

        response_dict = {
            "sensors": sensors,
            "count": count,
        }

        response = json.dumps(response_dict)

        return response

    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        mac_address = body_dict.get('mac_address', None)

        if mac_address is None:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request body.')

        try:
            redis_client.ts().create(f'{mac_address}:temperature', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        try:
            redis_client.ts().create(f'{mac_address}:humidity', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        return

## Sensor Endpoint Class

In [5]:
class Sensor(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')
        
        h_info = redis_client.ts().info(f'{mac_address}:humidity')

        response_dict = {
            "mac_address": mac_address,
            "t_samples": t_info.total_samples,
            "t_retention": t_info.retention_msecs,
            "h_samples": h_info.total_samples,
            "h_retention": h_info.retention_msecs,
        }

        response = json.dumps(response_dict)

        return response

    def PUT(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        t_retention = body_dict.get('t_retention', None)

        if t_retention is None:
            raise cherrypy.HTTPError(400, 'Missing temperature retention period in the request body.')

        h_retention = body_dict.get('h_retention', None)

        if h_retention is None:
            raise cherrypy.HTTPError(400, 'Missing humidity retention period in the request body.')

        redis_client.ts().alter(f'{mac_address}:temperature', retention_msecs=t_retention)
        redis_client.ts().alter(f'{mac_address}:humidity', retention_msecs=h_retention)

        return
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')
        
        mac_address = path[0]
        found = 0
        found += redis_client.delete(f'{mac_address}:temperature')
        found += redis_client.delete(f'{mac_address}:humidity')

        if found == 0:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        return

In [6]:
# HistoricalData Endpoint Class
class HistoricalData(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]
        start_date = query.get('start_date', None)
        end_date = query.get('end_date', None)

        if not start_date:
            raise cherrypy.HTTPError(400, 'Missing start date in the request parameters.')
        if not end_date:
            raise cherrypy.HTTPError(400, 'Missing end date in the request parameters.')

        try:
            start_timestamp = int(datetime.fromisoformat(start_date).timestamp() * 1000)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for start date in the request parameters.')

        try:
            end_timestamp = int(datetime.fromisoformat(end_date).timestamp() * 1000)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for end date in the request parameters.')

        if end_timestamp <= start_timestamp:
            raise cherrypy.HTTPError(400, 'End date smaller or equal than start date.')

        try:
            temperature_data = redis_client.ts().range(f'{mac_address}:temperature', start_timestamp, end_timestamp)
            humidity_data = redis_client.ts().range(f'{mac_address}:humidity', start_timestamp, end_timestamp)
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        timestamps = [int(data[0]) for data in temperature_data]
        temperatures = [int(data[1]) for data in temperature_data]
        humidities = [int(data[1]) for data in humidity_data]

        response_dict = {
            "mac_address": mac_address,
            "timestamp": timestamps,
            "temperature": temperatures,
            "humidity": humidities,
        }

        response = json.dumps(response_dict)
        return response

## Setup cherrypy and Map objects to their target endpoints

In [ ]:
conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Status(), '/status', conf)
cherrypy.tree.mount(Sensors(), '/sensors', conf)
cherrypy.tree.mount(Sensor(), '/sensor', conf)
cherrypy.tree.mount(HistoricalData(), '/data', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=09ce0a41-51ff-43bf-9b33-a2ff20a7fc7e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>